In [22]:
import tensorflow
tensorflow.random.set_seed(1)
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense


# Other essential libraries
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from numpy import array


In [23]:
n_timestamp = 10
train_days = 4000  # number of days to train from
testing_days = 500 # number of days to be predicted
n_epochs = 50
filter_on = 1

In [24]:
dataset = pd.read_csv('Colo.csv')
dataset.head()



,Date,Temp C
0,1/1/2009,31
1,2/1/2009,32
2,3/1/2009,31
3,4/1/2009,32
4,5/1/2009,31


In [25]:
train_set = dataset[0:train_days].reset_index(drop=True)
test_set = dataset[train_days: train_days+testing_days].reset_index(drop=True)
training_set = train_set.iloc[:, 1:2].values
testing_set = test_set.iloc[:, 1:2].values

In [26]:
sc = MinMaxScaler(feature_range = (0, 1))
training_set_scaled = sc.fit_transform(training_set)
testing_set_scaled = sc.fit_transform(testing_set)

In [27]:
# Split data into n_timestamp
def data_split(sequence, n_timestamp):
    X = []
    y = []
    for i in range(len(sequence)):
        end_ix = i + n_timestamp
        if end_ix > len(sequence)-1:
            break
        # i to end_ix as input
        # end_ix as target output
        seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
        X.append(seq_x)
        y.append(seq_y)
    return array(X), array(y)

In [28]:
X_train, y_train = data_split(training_set_scaled, n_timestamp)
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
X_test, y_test = data_split(testing_set_scaled, n_timestamp)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)

In [30]:
model = Sequential()
model.add(LSTM(200, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_3 (LSTM)                (None, 10, 200)           161600    
_________________________________________________________________
lstm_4 (LSTM)                (None, 50)                50200     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 211,851
Trainable params: 211,851
Non-trainable params: 0
_________________________________________________________________
